In [82]:
import os
from pprint import pprint
import dask.dataframe as dd
import matplotlib.pyplot as plt
import numpy as np
import aux as aux
from dask.distributed import SSHCluster,Client
import anomalies as an
import pandas as pd

In [114]:
client = Client("10.67.22.165:42429")

2024-07-05 08:01:32,343 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [138]:
path=os.path.abspath('/home/data/data.csv')

df = dd.read_csv(path)

In [139]:
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
# metrics = ['S41','S110','S100','S101','S102','S106','S107','S108','S122','S124','S126']
metrics = ['S112', 'S113','S114', 'S115','S123','S127','S201','S202', 'S203','S73']
# metrics = ['S112', 'S113', 'S114', 'S115',] #esterne

In [140]:
time, anom, val = an.anomaly_column(df, hardware_list[3], engine_list[0], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [141]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[3], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S112
Calculating for metric S113
Calculating for metric S114
Calculating for metric S115
Calculating for metric S123
Calculating for metric S127
Calculating for metric S201
Calculating for metric S202
Calculating for metric S203
Calculating for metric S73


In [142]:
def corr_finder(engine, metric_list, window):
    
    corr_coeff_pearson = []
    corr_coeff_spearman = []
    corr_coeff_kendall = []
    tot_corr_coeff = []

    for k in range(len(metric_list)):
        corr_coeff_pearson.append([])
        corr_coeff_spearman.append([])
        corr_coeff_kendall.append([])

    for metric in metric_list:
        dftot = pd.DataFrame({'values_engine': list(engine.iloc[:,1]), 'values_metric': list(metric.iloc[:,1])})
        corrs = [dftot.corr(method='pearson').iloc[0,1],dftot.corr(method='spearman').iloc[0,1],dftot.corr(method='kendall').iloc[0,1]]
        tot_corr_coeff.append(corrs)

    for i in range(len(engine['Anomaly'])):
        if engine.iloc[i,1]:
            values_engine = list(engine.iloc[i-window:i+window+1,2])
            for j,metric in enumerate(metric_list):
                values_metric = list(metric.iloc[i-window:i+window+1,1])

                df = pd.DataFrame({'values_engine': values_engine, 'values_metric': values_metric})
                corr_coeff_pearson[j].append(df.corr(method='pearson').iloc[0, 1])
                corr_coeff_spearman[j].append(df.corr(method='spearman').iloc[0, 1])
                corr_coeff_kendall[j].append(df.corr(method='kendall').iloc[0, 1])
    return corr_coeff_pearson,corr_coeff_spearman,corr_coeff_kendall,tot_corr_coeff

In [143]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [144]:
with open(f"{hardware_list[3]}_{engine_list[0]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [145]:
#115-S118
df = dd.read_csv(path)
time, anom, val = an.anomaly_column(df, hardware_list[3], engine_list[1], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [146]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [147]:
with open(f"{hardware_list[3]}_{engine_list[1]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [148]:
#088-S169

df = dd.read_csv(path)
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
# metrics = ['S41','S110','S100','S101','S102','S106','S107','S108','S122','S124','S126']
metrics = ['S130','S171','S172','S173','S174','S179','S183','S204','S205','S206']   #binary circ2

In [149]:
time, anom, val = an.anomaly_column(df, hardware_list[1], engine_list[2], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [150]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[1], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S130
Calculating for metric S171
Calculating for metric S172
Calculating for metric S173
Calculating for metric S174
Calculating for metric S179
Calculating for metric S183
Calculating for metric S204
Calculating for metric S205
Calculating for metric S206


In [151]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [152]:
with open(f"{hardware_list[1]}_{engine_list[2]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [153]:
#088-S170
df = dd.read_csv(path)
time, anom, val = an.anomaly_column(df, hardware_list[1], engine_list[3], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [154]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [155]:
with open(f"{hardware_list[1]}_{engine_list[3]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [156]:
#106-S169
df = dd.read_csv(path)
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
# metrics = ['S41','S110','S100','S101','S102','S106','S107','S108','S122','S124','S126']
metrics = ['S130','S171','S172','S173','S174','S179','S183','S204','S205','S206'] #binary circ2
# metrics = ['S112', 'S113', 'S114', 'S115',] #esterne

In [157]:
time, anom, val = an.anomaly_column(df, hardware_list[2], engine_list[2], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [158]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[2], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S130
Calculating for metric S171
Calculating for metric S172
Calculating for metric S173
Calculating for metric S174
Calculating for metric S179
Calculating for metric S183
Calculating for metric S204
Calculating for metric S205
Calculating for metric S206


In [159]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [160]:
with open(f"{hardware_list[2]}_{engine_list[2]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [161]:
#106-s170
df = dd.read_csv(path)
time, anom, val = an.anomaly_column(df, hardware_list[2], engine_list[3], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [162]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [163]:
with open(f"{hardware_list[2]}_{engine_list[3]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [164]:
#115-S169
df = dd.read_csv(path)
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
# metrics = ['S41','S110','S100','S101','S102','S106','S107','S108','S122','S124','S126']
metrics = ['S130','S171','S172','S173','S174','S179','S183','S204','S205','S206'] #binary circ2
# metrics = ['S112', 'S113', 'S114', 'S115',] #esterne

In [165]:
time, anom, val = an.anomaly_column(df, hardware_list[3], engine_list[2], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [166]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[3], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S130
Calculating for metric S171
Calculating for metric S172
Calculating for metric S173
Calculating for metric S174
Calculating for metric S179
Calculating for metric S183
Calculating for metric S204
Calculating for metric S205
Calculating for metric S206


In [167]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [168]:
with open(f"{hardware_list[3]}_{engine_list[2]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [169]:
#115-S170
df = dd.read_csv(path)
time, anom, val = an.anomaly_column(df, hardware_list[3], engine_list[3], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [170]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [171]:
with open(f"{hardware_list[3]}_{engine_list[3]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

### NON BINARY

In [172]:
#088-169
df = dd.read_csv(path)
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
# metrics = ['S41','S110','S100','S101','S102','S106','S107','S108','S122','S124','S126']
metrics = ['S157', 'S158','S159','S163','S164','S165','S178','S180']
# metrics = ['S112', 'S113', 'S114', 'S115',] #esterne

In [173]:
time, anom, val = an.anomaly_column(df, hardware_list[1], engine_list[2], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [174]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[1], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S157
Calculating for metric S158
Calculating for metric S159
Calculating for metric S163
Calculating for metric S164
Calculating for metric S165
Calculating for metric S178
Calculating for metric S180


In [175]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [176]:
with open(f"{hardware_list[1]}_{engine_list[2]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [177]:
#088-S170
df = dd.read_csv(path)
time, anom, val = an.anomaly_column(df, hardware_list[1], engine_list[3], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [178]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [179]:
with open(f"{hardware_list[1]}_{engine_list[3]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [180]:
#106-S169
df = dd.read_csv(path)
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
# metrics = ['S41','S110','S100','S101','S102','S106','S107','S108','S122','S124','S126']
metrics = ['S157', 'S158','S159','S163','S164','S165','S178','S180']
# metrics = ['S112', 'S113', 'S114', 'S115',] #esterne

In [181]:
time, anom, val = an.anomaly_column(df, hardware_list[2], engine_list[2], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [182]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[2], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S157
Calculating for metric S158
Calculating for metric S159
Calculating for metric S163
Calculating for metric S164
Calculating for metric S165
Calculating for metric S178
Calculating for metric S180


In [183]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [184]:
with open(f"{hardware_list[2]}_{engine_list[2]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [185]:
#106-S170
df = dd.read_csv(path)

time, anom, val = an.anomaly_column(df, hardware_list[2], engine_list[3], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [186]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [187]:
with open(f"{hardware_list[2]}_{engine_list[3]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [188]:
#115-169
df = dd.read_csv(path)
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
# metrics = ['S41','S110','S100','S101','S102','S106','S107','S108','S122','S124','S126']
metrics = ['S157', 'S158','S159','S163','S164','S165','S178','S180']
# metrics = ['S112', 'S113', 'S114', 'S115',] #esterne

In [189]:
time, anom, val = an.anomaly_column(df, hardware_list[3], engine_list[2], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [190]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[3], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S157
Calculating for metric S158
Calculating for metric S159
Calculating for metric S163
Calculating for metric S164
Calculating for metric S165
Calculating for metric S178
Calculating for metric S180


In [191]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [192]:
with open(f"{hardware_list[3]}_{engine_list[2]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [193]:
#115-S170
df = dd.read_csv(path)
time, anom, val = an.anomaly_column(df, hardware_list[3], engine_list[3], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [194]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [195]:
with open(f"{hardware_list[3]}_{engine_list[3]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [196]:
#065-169
df = dd.read_csv(path)
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
# metrics = ['S41','S110','S100','S101','S102','S106','S107','S108','S122','S124','S126']
metrics = ['S130','S171','S172','S173','S174','S179','S183','S204','S205','S206'] #binary circ2
# metrics = ['S112', 'S113', 'S114', 'S115',] #esterne

In [197]:
time, anom, val = an.anomaly_column(df, hardware_list[0], engine_list[2], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [198]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[0], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S130
Calculating for metric S171
Calculating for metric S172
Calculating for metric S173
Calculating for metric S174
Calculating for metric S179
Calculating for metric S183
Calculating for metric S204
Calculating for metric S205
Calculating for metric S206


In [199]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [200]:
with open(f"{hardware_list[0]}_{engine_list[2]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

In [201]:
#065-169
df = dd.read_csv(path)
hardware_list = ['SW-065', 'SW-088', 'SW-106', 'SW-115']

engine_list = ['S117', 'S118', 'S169', 'S170']
# metrics = ['S41','S110','S100','S101','S102','S106','S107','S108','S122','S124','S126']
metrics = ['S157', 'S158','S159','S163','S164','S165','S178','S180']
# metrics = ['S112', 'S113', 'S114', 'S115',] #esterne

In [202]:
time, anom, val = an.anomaly_column(df, hardware_list[0], engine_list[2], aggfunc='max')
engine_norm = pd.DataFrame({'Minute':time, 'Anomaly':anom, 'Value':val})

In [203]:
metric_list=[]

for metric in metrics:
    print(f"Calculating for metric {metric}")
    values, minutes = aux.get_norm_data(df, hardware_list[0], metric)
    metric_val = aux.fill_missing_min(values, minutes)
    metric_norm = pd.DataFrame({'Minute':time, 'Value':metric_val})
    metric_list.append(metric_norm)

Calculating for metric S157
Calculating for metric S158
Calculating for metric S159
Calculating for metric S163
Calculating for metric S164
Calculating for metric S165
Calculating for metric S178
Calculating for metric S180


In [204]:
pearson,spearman,kendall,tot=corr_finder(engine_norm,metric_list,window=50)

In [205]:
with open(f"{hardware_list[0]}_{engine_list[2]}_correlations.txt",'a') as file:
    for i,metric in enumerate(metrics):
        file.write(f'Correlation coefficients for metric {metric}')
        file.write('\n\n')
        file.write(f'Pearson, Spearman and Kendall total correlations:')
        file.write(' '.join(map(str,tot[i])))
        file.write('\n\n')
        file.write(f'Pearson coefficients: ')
        file.write(' '.join(map(str,pearson[i])))
        file.write('\n')
        file.write(f'Spearman coefficients: ')
        file.write(' '.join(map(str,spearman[i])))
        file.write('\n')
        file.write(f'Kendall coefficients: ')
        file.write(' '.join(map(str,kendall[i])))
        file.write('\n\n')

: 

_____________________________________________________

In [43]:
df_fourmotors=df[df['metric'].isin(['S117','S118','S169','S170'])]

In [29]:
sel=df[df['metric'] == 'S7'].compute()
#sel= sel[sel['hwid'] == 'SW-088']
#sel[sel['value']!=0].compute().head()


In [30]:
sel['value'].unique()

array([0, 1])

In [11]:
sel.head()

when    hwid metric  value
57   1601510485159  SW-065    S40    133
143  1601510515180  SW-065    S40    134
283  1601510545278  SW-065    S40    134
469  1601510575516  SW-065    S40    135
582  1601510607617  SW-065    S40    135

In [32]:
# sel = sel[sel['value'] != 2]
sel[sel['value'] != 0].head()


when    hwid metric  value
424030  1601510458920  SW-088     S7      1
424168  1601510487842  SW-088     S7      1
424301  1601510519381  SW-088     S7      1
424436  1601510548722  SW-088     S7      1
424619  1601510579582  SW-088     S7      1

In [12]:
sel['value'].unique()

array([133, 134, 135, 136, 137, 138, 130, 128, 126, 124, 122, 121, 119,
       118, 117, 116, 115, 114, 113, 127, 131, 132, 123, 112, 111, 110,
       125, 109, 129, 120, 139, 108, 107, 106, 142, 144, 145, 146, 147,
       148, 149, 140, 141, 143, 105, 104, 151, 150, 103, 102, 101, 100,
       152,   0,  99,  98,  97,  96,  83,  82,  78,  91,  89,  84,  80,
        77,  73,  71,  70,  67,  69,  63,  62,  61,  60,  59,  55,  54,
        68,  86,  87,  88,  81,  85,  75,  66,  65,  58,  49,  79,  90,
        92,  93,  94,  95, 153, 157, 154, 155, 158, 159, 160, 161, 162,
       163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175,
       176, 177, 178, 179, 180, 182, 183, 184, 185, 186, 187, 188, 189,
       190, 191, 192, 193, 194, 195, 197, 199, 196, 200, 202, 203, 205,
       207, 209, 212, 214, 216, 218, 220, 222, 225,  64,  76,  72,  74,
        57,  56,  53,  52,  51,  50,  48,  47,  46,  44,  45, 310, 312,
       307, 304, 315, 317, 320, 322, 325, 327, 330, 333, 335, 34